In [10]:
!pip install pandas numpy scikit-learn sentence-transformers faiss-cpu openai pyngrok streamlit


In [11]:
import pandas as pd

df = pd.read_csv("Training Dataset.csv")
df.fillna("NA", inplace=True)
df.head()



/tmp/ipython-input-11-2441670009.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("NA", inplace=True)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NA,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [12]:
import pandas as pd

df = pd.read_csv("Training Dataset.csv")
df.fillna("NA", inplace=True)
df.head()

documents = []
for index, row in df.iterrows():
    doc = f"Applicant {row['Loan_ID']} with gender {row['Gender']} and income {row['ApplicantIncome']} requested a loan of {row['LoanAmount']}. Loan Status: {row['Loan_Status']}"
    documents.append(doc)

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and fast

# Convert documents to embeddings
embeddings = model.encode(documents)

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

def search(query, k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [documents[i] for i in indices[0]]

import openai
openai.api_key = "your_openai_api_key"

def generate_response(context, query):
    prompt = f"Use the following information to answer:\n{context}\n\nQuestion: {query}\nAnswer:"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

def rag_qa_bot(query):
    relevant_docs = search(query)
    context = "\n".join(relevant_docs)
    answer = generate_response(context, query)
    return answer

import streamlit as st

st.title("📊 Loan Dataset RAG Q&A Chatbot")

user_query = st.text_input("Ask a question about the loan dataset:")

if st.button("Get Answer"):
    with st.spinner("Searching and thinking..."):
        answer = rag_qa_bot(user_query)
        st.success(answer)

import os
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")


SyntaxError: unmatched ')' (ipython-input-12-3280634250.py, line 62)

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2zp1XXdGx3m8SKJI6lCqmqerY2P_8tEh8YtB2xyEvKiq2s6y")


In [ ]:
from pyngrok import ngrok
import threading

with open("app.py", "w") as f:
    f.write(open("RAG_Q&A_Chatbot.py").read())

def run():
    os.system("streamlit run RAG_Q&A_Chatbot.py")

thread = threading.Thread(target=run)
thread.start()

public_url = ngrok.connect(8501)
print("🔗 Open your chatbot here:", public_url)
